https://jira.x5.ru/browse/CVMUPG-1089

In [1]:
name = 'CVMUPG-1089_kimberly_add'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/cvm_upgrade/')
sys.path.append('/home/jovyan/cvm_upgrade/upgrade/')
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
from upgrade.post_analysis2.tools.groups_metrics_aggregation import GroupsMetricsAggregation
from upgrade.post_analysis2.tools.guests_metrics_preprocessing import GuestsMetricsPreprocessing
from upgrade.post_analysis2.tools.post_analysis_params import PostAnalysisParams
from upgrade.post_analysis2.tools.metric_names_mapper import map_col_names_to_russian
from upgrade.post_analysis2.tools.experiment_participants import ExperimentParticipants
from upgrade.post_analysis2.tools.experiment_params import ExperimentParams
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/11 11:03:12 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

#### Найдем топ-5 брендов-конкурентов и возьмем их PLU.

In [5]:
start_date = datetime.date(2021, 11, 8)
end_date = datetime.date(2022, 2, 7)

In [6]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [7]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter(F.col('loyalty_cardholder_age_yrs') < 40)
                        .filter(F.col('gender_dk') == 'F')
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')

In [8]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [9]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                                , 'zsalnovat'
                               )
                   )


In [10]:
plu_hierarchy_lvl_4_dk = ['FD0911001']
# syntethic_category_id = [96,57,75,73,101,55,56,22]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [11]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [12]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [13]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [14]:
checks_tc5 = (checks_tc5.join(spark.table(PRODUCTS)
                             .withColumnRenamed('plu_id', 'plu_code')
                             .select('plu_code', 'plu_brand_code', 'plu_brand_hierarchy_lvl_nm'), 'plu_code', 'inner'))

In [15]:
columns = ['plu_brand_code', 'plu_brand_hierarchy_lvl_nm']
pdf = checks_tc5.groupby(columns).agg(F.sum('zsalnovat').alias('sum'))
pdf = pdf.orderBy('sum', ascending=False).limit(6)

In [16]:
brands = pdf.toPandas()

In [17]:
brands

,plu_brand_code,plu_brand_hierarchy_lvl_nm,sum
0,A411,ALWAYS,61837387.55
1,2452,KOTEX,43398672.91
2,2639,LIBRESSE,32645663.08
3,3137,NATURELLA,29654491.42
4,1247,DISCREET,23778451.04
5,3232,O.B.,22277678.41


In [18]:
brands = brands.drop(index=1)

In [19]:
brands['plu_brand_code'].to_list()

['A411', '2639', '3137', '1247', '3232']

In [20]:
plu_codes = (spark.table("hive_ssa_tc5.cvm_product")
                 .filter(F.col('plu_hierarchy_lvl_4_dk') == 'FD0911001')
                 .filter(F.col('plu_brand_code').isin(brands['plu_brand_code'].to_list()))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist())

In [21]:
len(plu_codes)

319

In [60]:
brand_plu = (spark.table("hive_ssa_tc5.cvm_product")
                 .filter(F.col('plu_hierarchy_lvl_4_dk') == 'FD0911001')
                 .filter(F.col('plu_brand_code').isin(brands['plu_brand_code'].to_list()))
                 .select('plu_id', 'plu_nm', 'plu_brand_hierarchy_lvl_nm')
                 .distinct()
                 .toPandas())

In [62]:
brand_plu = brand_plu.sort_values('plu_brand_hierarchy_lvl_nm')

In [99]:
val = brand_plu.reset_index(drop=True)

In [100]:
val.head()

,plu_id,plu_nm,plu_brand_hierarchy_lvl_nm
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS


### A-24-17-recurrent-Kimberly-clark-brand-w2-no

In [67]:
camp_id = 'A-24-17-recurrent-Kimberly-clark-brand-w2-no'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        65307: "sms_viber_tg",
        63797: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-20',
    'date_communication_to': '2021-10-26',

    'wave_ids': [
        65380 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [33]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [87]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [ ]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [88]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [89]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [102]:
val.columns = ['plu_code', 'plu_nm', 'plu_brand_hierarchy_lvl_nm']

In [ ]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

In [108]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.000,4.000,4.000,7.000,3.000,7.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.000,NaN,2.000,NaN,1.000
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.000,NaN,3.000,NaN,9.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN


### A-24-17-recurrent-Kimberly-clark-brand-w2-yes

In [112]:
camp_id = 'A-24-17-recurrent-Kimberly-clark-brand-w2-yes'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        65300: "sms_viber_tg",
        63797: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-20',
    'date_communication_to': '2021-10-26',

    'wave_ids': [
        65365 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [113]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [33]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [115]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [116]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [117]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [118]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [120]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [121]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.000,4.000,4.000,7.000,3.000,7.000,3.000,4.000,6.000,7.000,3.000,7.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.000,NaN,2.000,NaN,1.000,1.000,1.000,NaN,2.000,1.000,1.000
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.000,NaN,3.000,NaN,9.000,1.000,1.000,6.000,3.000,NaN,9.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### A-24-16-recurrent-Kimberly-clark-all-w2-no

In [122]:
camp_id = 'A-24-16-recurrent-Kimberly-clark-all-w2-no'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        65289: "sms_viber_tg",
        63796: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-20',
    'date_communication_to': '2021-10-26',

    'wave_ids': [
        65401 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [123]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [33]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [125]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [126]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [ ]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [128]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [129]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [130]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.000,4.000,4.000,7.000,3.000,7.000,3.000,...,6.000,7.000,3.000,7.000,3.000,8.000,8.000,11.000,14.000,20.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.000,NaN,2.000,NaN,1.000,1.000,...,NaN,2.000,1.000,1.000,4.000,1.000,NaN,2.000,NaN,NaN
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.000,NaN,3.000,NaN,9.000,1.000,...,6.000,3.000,NaN,9.000,4.000,4.000,2.000,8.000,9.000,5.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### A-24-16-recurrent-Kimberly-clark-all-w2-yes

In [131]:
camp_id = 'A-24-16-recurrent-Kimberly-clark-all-w2-yes'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        65291: "sms_viber_tg",
        63796: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-20',
    'date_communication_to': '2021-10-26',

    'wave_ids': [
        65398 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [132]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [33]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [134]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [135]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [136]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [137]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [138]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [139]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.000,4.000,4.000,7.000,3.000,7.000,3.000,...,8.000,11.000,14.000,20.000,6.000,8.000,9.000,11.000,5.000,20.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.000,NaN,2.000,NaN,1.000,1.000,...,NaN,2.000,NaN,NaN,NaN,1.000,NaN,2.000,1.000,NaN
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.000,NaN,3.000,NaN,9.000,1.000,...,2.000,8.000,9.000,5.000,3.000,4.000,5.000,8.000,5.000,5.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### X-24-18-KimberlyClark-Kotex-Competitors-2-sof

In [140]:
camp_id = 'X-24-18-KimberlyClark-Kotex-Competitors-2-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63969: "sms_viber_tg",
        63970: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-07',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        64054 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [141]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [33]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [143]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [144]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [145]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

22/02/10 21:54:22 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.57:39828 is closed
22/02/10 21:54:22 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.57:38312 is closed
22/02/10 21:54:22 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.57:36898 is closed
22/02/10 21:54:22 ERROR client.TransportClient: Failed to send RPC RPC 4998712045845956390 to /192.168.234.57:36898: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)


In [146]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [147]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [148]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.000,4.000,4.000,7.000,3.000,7.000,3.000,...,9.000,11.000,5.000,20.000,162.000,51.000,211.000,52.000,159.000,38.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.000,NaN,2.000,NaN,1.000,1.000,...,NaN,2.000,1.000,NaN,14.000,6.000,28.000,3.000,22.000,3.000
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.000,NaN,3.000,NaN,9.000,1.000,...,5.000,8.000,5.000,5.000,129.000,32.000,161.000,34.000,108.000,24.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### X-24-19-KimberlyClark-Kotex-Loyal-2-sof

In [149]:
camp_id = 'X-24-19-KimberlyClark-Kotex-Loyal-2-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63965: "sms_viber_tg",
        63782: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-07',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        64055 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [150]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [33]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [152]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [153]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [154]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [155]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [156]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [157]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.000,4.000,4.000,7.000,3.000,7.000,3.000,...,211.000,52.000,159.000,38.000,11.000,54.000,11.000,62.000,1.000,53.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.000,NaN,2.000,NaN,1.000,1.000,...,28.000,3.000,22.000,3.000,NaN,7.000,NaN,4.000,2.000,5.000
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.000,NaN,3.000,NaN,9.000,1.000,...,161.000,34.000,108.000,24.000,5.000,43.000,12.000,65.000,4.000,37.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### A-24-17-recurrent-Kimberly-clark-brand

In [158]:
camp_id = 'A-24-17-recurrent-Kimberly-clark-brand'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63795: "sms_viber_tg",
        63797: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        65380 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [159]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [33]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [161]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [162]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [163]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [164]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [165]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [166]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.000,4.000,4.000,7.000,3.000,7.000,3.000,...,11.000,62.000,1.000,53.000,8.000,7.000,7.000,8.000,7.000,8.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.000,NaN,2.000,NaN,1.000,1.000,...,NaN,4.000,2.000,5.000,NaN,2.000,1.000,1.000,1.000,1.000
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.000,NaN,3.000,NaN,9.000,1.000,...,12.000,65.000,4.000,37.000,7.000,6.000,2.000,2.000,3.000,2.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### A-24-16-recurrent-Kimberly-clark-all

In [167]:
camp_id = 'A-24-16-recurrent-Kimberly-clark-all'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63794: "sms_viber_tg",
        63796: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        63866 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [168]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [33]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [170]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [171]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [172]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [173]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [174]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [175]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.000,4.000,4.000,7.000,3.000,7.000,3.000,...,7.000,8.000,7.000,8.000,19.000,14.000,4.000,2.000,18.000,12.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.000,NaN,2.000,NaN,1.000,1.000,...,1.000,1.000,1.000,1.000,2.000,2.000,NaN,NaN,4.000,2.000
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.000,NaN,3.000,NaN,9.000,1.000,...,2.000,2.000,3.000,2.000,8.000,8.000,2.000,3.000,14.000,7.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### X-24-16-KimberlyClark-Kotex-Competitors-sof

In [176]:
camp_id = 'X-24-16-KimberlyClark-Kotex-Competitors-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63780: "sms_viber_tg",
        63785: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        63873 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [177]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [33]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [179]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [180]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [181]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [182]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [183]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [184]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.000,4.000,4.000,7.000,3.000,7.000,3.000,...,4.000,2.000,18.000,12.000,268.000,118.000,385.000,156.000,216.000,99.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.000,NaN,2.000,NaN,1.000,1.000,...,NaN,NaN,4.000,2.000,40.000,19.000,43.000,19.000,23.000,14.000
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.000,NaN,3.000,NaN,9.000,1.000,...,2.000,3.000,14.000,7.000,178.000,71.000,299.000,126.000,153.000,54.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### X-24-15-KimberlyClark-Kotex-Loyal-sof

In [26]:
camp_id = 'X-24-15-KimberlyClark-Kotex-Loyal-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63771: "sms_viber_tg",
        63782: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        63870 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [27]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [28]:
if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
              .select('customer_rk').distinct()
             )
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
              .select('customer_rk').distinct()
             )

In [29]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [ ]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [ ]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [ ]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [ ]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

In [ ]:
val.head()

In [212]:
val.to_csv('val.csv', index=False)

In [22]:
val = pd.read_csv('val.csv')

### 11: f24_freq24_Kimberly_Clark

In [31]:
camp_id = 'f24_freq24_Kimberly_Clark'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63744: "sms_viber_tg",
        63744: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        -1 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

[Stage 21:(114 + 106) / 3915][Stage 22:(125 + 19) / 210][Stage 23:> (0 + 0) / 150]

In [32]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

[Stage 21:(130 + 106) / 3915][Stage 22:(134 + 19) / 210][Stage 23:> (0 + 0) / 150]

In [33]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [34]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

[Stage 21:(133 + 106) / 3915][Stage 22:(135 + 19) / 210][Stage 23:> (0 + 0) / 150]

In [37]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

[Stage 74:>                 (0 + 5) / 6][Stage 75:>                 (0 + 0) / 1]00]]]]]22/02/11 11:39:22 ERROR client.TransportClient: Failed to send RPC RPC 5934918335446525503 to /192.168.234.57:51036: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
22/02/11 11:39:22 ERROR cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Sending RequestExecutors(2,7,Map(mn-hd0188.x5.ru -> 1, mn-hd0225.x5.ru -> 1, mn-hd0331.x5.ru -> 1, mn-hd0344.x5.ru -> 1, mn-hd0016.x5.ru -> 1, mn-hd0154.x5.ru -> 1, mn-hd0181.x5.ru -> 1, mn-hd0020.x5.ru -> 1, mn-hd0104.x5.ru -> 1, mn-hd0287.x5.ru -> 1, mn-hd0262.x5.ru -> 1, mn-hd0111.x5.ru -> 1, mn-hd0276.x5.ru -> 1, mn-hd0321.x5.ru -> 1, mn-hd0207.x5.ru -> 1, mn-hd0289.x5.ru -> 1, mn-hd0337.x5.ru -> 2, mn-hd0132.x5.ru -> 1, mn-hd0061.x5.ru -> 1, mn-hd0240.x5.ru -> 1),Set()) to AM was unsuccessful
java.io.IOException: Failed to send RPC RPC 593491833544

In [38]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [39]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [47]:
val['plu_code'] = val['plu_code'].astype('str')

In [48]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [49]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x.1,qty_y.1,qty_x.2,qty_y.2,qty_x.3,...,qty_x.28,qty_y.28,qty_x.29,qty_y.29,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.00,4.00,4.00,7.00,3.00,7.00,3.00,...,161.00,70.00,136.00,53.00,77.000,63.000,324.000,224.000,71.000,57.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.00,NaN,2.00,NaN,1.00,1.00,...,15.00,5.00,12.00,5.00,28.000,14.000,58.000,31.000,12.000,13.000
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.00,NaN,3.00,NaN,9.00,1.00,...,146.00,64.00,82.00,37.00,56.000,59.000,265.000,207.000,44.000,48.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### U-24-12-kotex-loyal-viber-sms

In [50]:
camp_id = 'U-24-12-kotex-loyal-viber-sms'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63779: "sms_viber_tg",
        63783: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        63865 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [51]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [52]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [53]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [54]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [55]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [56]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [57]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [58]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x.1,qty_y.1,qty_x.2,qty_y.2,qty_x.3,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.00,4.00,4.00,7.00,3.00,7.00,3.00,...,324.000,224.000,71.000,57.000,303.000,162.000,349.000,199.000,262.000,130.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.00,NaN,2.00,NaN,1.00,1.00,...,58.000,31.000,12.000,13.000,24.000,7.000,16.000,11.000,22.000,11.000
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.00,NaN,3.00,NaN,9.00,1.00,...,265.000,207.000,44.000,48.000,204.000,109.000,255.000,141.000,131.000,76.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### U-24-12-kotex-competitors-viber-sms

In [59]:
camp_id = 'U-24-12-kotex-competitors-viber-sms'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63760: "sms_viber_tg",
        63769: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        63865 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [60]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [61]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [62]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [63]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [64]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [65]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [66]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [67]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x.1,qty_y.1,qty_x.2,qty_y.2,qty_x.3,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,3639593,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS,2.00,4.00,4.00,7.00,3.00,7.00,3.00,...,349.000,199.000,262.000,130.000,421.000,118.000,475.000,128.000,394.000,110.000
1,3932119,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALWAYS,NaN,1.00,NaN,2.00,NaN,1.00,1.00,...,16.000,11.000,22.000,11.000,47.000,19.000,56.000,16.000,44.000,23.000
2,3359506,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3637887,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS,NaN,1.00,NaN,3.00,NaN,9.00,1.00,...,255.000,141.000,131.000,76.000,278.000,74.000,386.000,129.000,219.000,69.000
4,3393598,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALWAYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
list_columns = ['plu_code',
'plu_nm', 'plu_brand_hierarchy_lvl_nm',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-16-recurrent-Kimberly-clark-all',
'A-24-16-recurrent-Kimberly-clark-all',
'A-24-16-recurrent-Kimberly-clark-all',
'A-24-16-recurrent-Kimberly-clark-all',
'A-24-16-recurrent-Kimberly-clark-all',
'A-24-16-recurrent-Kimberly-clark-all',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'f24_freq24_Kimberly_Clark',
'f24_freq24_Kimberly_Clark',
'f24_freq24_Kimberly_Clark',
'f24_freq24_Kimberly_Clark',
'f24_freq24_Kimberly_Clark',
'f24_freq24_Kimberly_Clark',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-competitors-viber-sms',
'U-24-12-kotex-competitors-viber-sms',
'U-24-12-kotex-competitors-viber-sms',
'U-24-12-kotex-competitors-viber-sms',
'U-24-12-kotex-competitors-viber-sms',
'U-24-12-kotex-competitors-viber-sms']

In [72]:
for i in range(3, 79, 6):
    list_columns[i] += '_period_tg'
    list_columns[i+1] += '_period_cg'
    list_columns[i+2] += '_pre_tg'
    list_columns[i+3] += '_pre_cg'
    list_columns[i+4] += '_post_tg'
    list_columns[i+5] += '_post_cg'

In [74]:
val.columns = list_columns

In [75]:
val.iloc[:, 3:] = val.iloc[:, 3:].astype('float')

In [76]:
val = val.T

In [77]:
val

,0,1,2,3,4,5,6,7,8,9,...,309,310,311,312,313,314,315,316,317,318
plu_code,3639593,3932119,3359506,3637887,3393598,44433,3636902,70525,3171371,3932508,...,3490586,53533,53373,3425801,3945039,19493,3244049,3197274,3367588,2084925
plu_nm,ALWAYS Прокладки PLAT.ULT.NORM.PL.гиг8шт,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.ежед.ар.16шт,ALW.Прокл.Ultra Norm.Plus 10шт+Nght 7шт,ALWAYS Проклад.PLAT.ULT.SUP.PL.гиг.14шт,ALWAYS Прокладки ULTR.SENS.NIG.ж.гиг.7шт,ALW.Прокладки CLASSIC 10шт,ALWAYS Прокл.К.ДЕНЬ FR&PR.Норм.Trio 60шт,ALWAYS Прокладки ULTRA NORMAL 20шт,ALW.Прокадки CLAS.NORM.SENS10шт,ALWAYS Прокл.НЕЗАМ.ЗАЩ.УДЛ.гиг.ар.28шт,...,O.B.Тампоны PROCOMF.NIGHT SUP+COMF.16шт,O.B.Тампоны PRO COMFORT 16шт,O.B.Тампоны МИНИ comfort 8шт,OB Тампон NORMAL c аппликатором 16шт,O.B.Тампоны EXTRADEFENCE NORMAL 16шт,O.B.Тампоны PROCOMF.мини 16шт,OB Тампоны ПРО КОМФОРТ нормал 8шт,Тампоны OB OPTIBALANCE нормал 16шт,O.B.Тампоны Procomfort супер плюс 16шт,O.B.Тампоны PROCOMF.пр/уп.8+8шт
plu_brand_hierarchy_lvl_nm,ALWAYS,ALWAYS,ALWAYS,ALWAYS,ALWAYS,ALWAYS,ALWAYS,ALWAYS,ALWAYS,ALWAYS,...,O.B.,O.B.,O.B.,O.B.,O.B.,O.B.,O.B.,O.B.,O.B.,O.B.
A-24-17-recurrent-Kimberly-clark-brand-w2-no_period_tg,2.00,NaN,NaN,NaN,NaN,NaN,NaN,7.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-no_period_cg,4.00,1.00,NaN,1.00,NaN,NaN,NaN,20.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-no_pre_tg,4.00,NaN,NaN,NaN,NaN,NaN,NaN,3.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-no_pre_cg,7.00,2.00,NaN,3.00,NaN,NaN,NaN,54.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.00,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-no_post_tg,3.00,NaN,NaN,NaN,NaN,NaN,NaN,11.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-no_post_cg,7.00,1.00,NaN,9.00,NaN,NaN,NaN,26.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-yes_period_tg,3.00,1.00,NaN,1.00,NaN,NaN,NaN,8.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,NaN


In [78]:
val.to_excel('all_camp_plu_against_1' +'.xlsx')

In [79]:
val = val.T

In [81]:
val.to_excel('all_camp_plu_against_2' +'.xlsx')